In [10]:
#define path
wesley_ds = "/files"
kaggle_ds = "/archive/Data/genres_original"
main_path = "Dataset"

In [11]:
#choose the dataset
chooser = wesley_ds

In [12]:
DATASET_PATH = main_path+chooser
# DATASET_PATH = "../../../Music"
JSON_PATH = 'complete_3_first.json'
SAMPLE_RATE = sr =  22050
DURATION = 30 #measured in seconds 
SAMPLES_PER_TRACK = SAMPLE_RATE*DURATION

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(main_path+chooser):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        print(filename)
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [1]:
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import soundfile as sf
# import librosa
# import librosa.display
from IPython.display import Audio
# from tqdm import tqdm 
from sklearn.model_selection import train_test_split, TimeSeriesSplit
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, LSTM, Bidirectional, GRU, BatchNormalization, LeakyReLU
# from tensorflow.keras.utils import to_categorical
# import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import math
import json
import random
# import mutagen

In [ ]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=4084, hop_length=1024, num_segments=10):
    #dictionary to store data
    data = {
        'mapping' : [],
        'mfcc' : [],
        'labels' : [],
        'energy': [],
        'key': []
    }
    
    count = 0 # To keep track of our progress
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments) 
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)
    
    #Loop through all the genres
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        #ensure that we're not at the root level
        if dirpath not in dataset_path:

            #save the semantic label
            dirpath_components = dirpath.split('/')
            semantic_label = dirpath_components[-1]
            data['mapping'].append(semantic_label)
            print('\nProcessing {}'.format(semantic_label))
            
            #process files for a specific genre 
            for f in filenames:
#                 if f.endswith('.wav') and f != 'jazz.00054.wav': # Since file jazz.00054.wav is an empty file
                if f.endswith('.mp3') or f.endswith('.MP3'): # Since file jazz.00054.wav is an empty file
                    file_path = os.path.join(dirpath,f)
                    
                    #loading the audio file 
                    # we are using the soundfile library since it is faster than librosa
                    signal, sr = librosa.load(file_path) # len(signal) = 661794  # sr is 22050 by default
                    try:
                        temp1 = str(mutagen.File(file_path)['TXXX:EnergyLevel'])
                    
                        temp2 = str(mutagen.File(file_path)['TKEY'])
                    except:
                        continue
                    midptr = int(signal.shape[0]/2)
                    bptr = midptr - (SAMPLE_RATE*int(DURATION/2)) #set as OFFSET
#                     hptr = midptr + (SAMPLE_RATE*duration)
                    #print(signal,sr)
                    #process segments extracting mfcc and storing data
                    for s in range(num_segments): 
                        # Since num_segments is defined as 5. Every 30 sec file is divided into 5 segments of length 6sec 
                        # Start sample would keep track of the index of the first element of each 6 second batch
                        # finish sample would keep track of the index of the last element of each 6 second batch
                        # And then with the help of python's slice functionality we will extract that 6 second batch from every 30 sec signal
                        start_sample = bptr + num_samples_per_segment * s   
                        finish_sample = num_samples_per_segment + start_sample
                        
                        # Next, we will pass each segment in order to extract MFCC. The parameter n_mfcc defines the number of mfcc 
                        # we need to extract, Usually n_mfcc is set b/w 13 to 40. The other parameters n_fft and hop length are 
                        # indivisual topics of discussion. Will be discussed in later Notbooks. 
                        mfcc = librosa.feature.mfcc(signal[start_sample : finish_sample],
                                                   sr = sr,
                                                   n_fft = n_fft,
                                                   n_mfcc = n_mfcc,
                                                   hop_length = hop_length)

                        mfcc = mfcc.T
                        # store mfcc for segment if it has the expected length
                        if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                            print(mfcc.shape)
                            data['mfcc'].append(mfcc.tolist())
                            data['labels'].append(i)
                            data['energy'].append(temp1)
                            data['key'].append(temp2)
                            print('Processing {}, segment:{}'.format(file_path, s))
                            count += 1
                            print(count)
    with open(json_path, 'w') as fp:
        json.dump(data, fp, indent=4)

In [ ]:
# Let's run the above function 
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

In [2]:
# loading the saved Json file
def load_data(path):
    with open(path, 'r') as fp:
        data = json.load(fp)
        
    #Convert lists into numpy arrays
    inputs = data['mfcc']
    targets = data['labels'] 
    return np.array(inputs), np.array(targets)

In [ ]:
np.unique(targets, return_counts=True)

In [ ]:
#for key purpose
targets2 = np.array([outp_key.index(isi) for isi in targets])
print(targets2)

for isi in targets:
    print(outp_key[outp_key.index(isi)], isi)

In [ ]:
print(np.unique(targets2, return_counts=True))
print(len(outp_key))

In [ ]:
# Keras Model

In [ ]:
model = Sequential()


model.add(GRU(600, input_shape=(inputs.shape[1], inputs.shape[2])))

# model.add(GRU(100, return_sequences=True, input_shape=(inputs.shape[1], inputs.shape[2])))
# model.add(GRU(200, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())
# model.add(GRU(500))

model.add(Dropout(0.4))
# model.add(BatchNormalization())

model.add(Dense(24, 'softmax'))


model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])



model.summary()

In [ ]:
history = model.fit(inputs_train, targets_train,
          validation_data=(inputs_test, targets_test),
          epochs = 35,
          batch_size=100)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
# Pytorch

In [21]:
def augment_ds(inputs, outputs):
    inputs_cpy = np.array(inputs)
    inputs_ori = np.array(inputs)
    outputs_cpy = np.array(outputs)
    # Adding Noise 
    
    for i in range(inputs_cpy.shape[0]):
        s = np.random.rand(inputs_cpy.shape[1], inputs_cpy.shape[2])
        inputs_cpy[i] = inputs_cpy[i] + s

    #join dataset
    inputs_cpy = np.vstack((inputs_cpy, inputs_ori))
    outputs_cpy = np.hstack((outputs, outputs))
    print(inputs_cpy.shape, outputs_cpy.shape)
    return inputs_cpy, outputs_cpy

In [22]:
# Model Definition
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, prob_do):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.do = nn.Dropout(p=prob_do)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.gru(x, h0)  
        out = out[:, -1, :]
        out = self.do(out)
        out = torch.sigmoid(self.fc(out))
        return out

In [23]:
def create_models(input_size, hidden_size, num_classes, device, prob_do = 0.5, num_layers = 1):
    return RNN(input_size, hidden_size, num_layers, num_classes, prob_do).to(device)

In [24]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

def train_models(num_epoch, batch_size, learning_rate, inputs, outputs, splitts, device, folds = 0):
    train_index, test_index = splitts[folds]
    x_train = inputs[train_index]
    y_train = outputs[train_index]
    
    x_train, y_train = augment_ds(x_train, y_train)
    x_test = inputs[test_index]
    y_test = outputs[test_index]
    
    models = create_models(inputs.shape[-1], 600, 5, device, prob_do = 0.4)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(models.parameters(), lr=learning_rate)
    
    x_train = torch.from_numpy(x_train)
    y_train = torch.from_numpy(y_train)
    x_test = torch.from_numpy(x_test)
    y_test = torch.from_numpy(y_test)
#     x_val = torch.from_numpy(x_val)
#     y_val = torch.from_numpy(y_val)
    
    print(x_train.shape)
    train_loss = []
    test_loss = []
    acc_arr = []
    train = TensorDataset(x_train, y_train)
    test = TensorDataset(x_test, y_test)
#     val = TensorDataset(x_val, y_val)
    
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)
#     val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)
#     test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)
    for epoch in range(num_epoch):
        # training sequences
        batch_losses = []
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.view([batch_size, -1, inputs.shape[-1]]).to(device).to(torch.float32)
            y_batch = y_batch.to(device).to(torch.int64)
            
            models.train()
            y_hat = models(x_batch)
            loss = criterion(y_hat, y_batch)

            # Backward and optimize
            loss.backward()
            # Updates parameters and zeroes gradients
            optimizer.step()
            optimizer.zero_grad()
            
            batch_losses.append(loss.item())
        train_loss.append(np.mean(batch_losses))
        
        #testing sequences
        with torch.no_grad():
            batch_val_losses = []
            for x_val, y_val in test_loader:
                x_val = x_val.view([batch_size, -1, inputs.shape[-1]]).to(device).to(torch.float32)
                y_val = y_val.to(device).to(torch.int64)
                models.eval()
                yhat = models(x_val)
                val_loss = criterion(yhat, y_val)
                batch_val_losses.append(val_loss.item())
            test_loss.append(np.mean(batch_val_losses))
        
        print(f"[{epoch+1}/{num_epoch}] Training loss: {train_loss[-1]:.4f}\t Validation loss: {test_loss[-1]:.4f}", end="\n")
            
    return models,train_loss, test_loss, acc_arr

In [25]:
def evaluation(models, test_loader,device, batch_size):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        batch_acc = []
        #labels = labels.to(device)
        for x_test, y_test in test_loader:
            x_test = x_test.view([batch_size, -1, x_test.shape[-1]]).to(device)
            y_test = y_test.to(device)
            x_test = x_test.to(torch.float32)
            y_test = y_test.to(torch.int64)
            models.eval()  
            outputs = models(x_test)
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += y_test.size(0)
#             print(y_test)
            n_correct += (predicted == y_test).sum().item()
            acc = 100.0 * n_correct / n_samples
            batch_acc.append(acc)
#         print(n_correct)
        print(f'Accuracy: {np.mean(batch_acc) :.4f} %')

In [13]:
inputs, targets = load_data(JSON_PATH)

In [27]:
#device conf
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 5 #deprecated
num_epochs = 30
batch_size = 100
learning_rate = 0.001

hidden_size = 500 #deprecated
num_layers = 1 #deprecated

from sklearn.model_selection import KFold as StratifiedKFold
# manage dataset

# split training and testing
x_train, x_val, y_train, y_val = train_test_split(inputs, targets, test_size=0.1)
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)
val = TensorDataset(x_val, y_val)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

skf = StratifiedKFold(n_splits=10)
output_lst = ["BASS HOUSE", "BIG ROOM", "DEEP HOUSE", "ELECTRO HOUSE", "FUTURE HOUSE"]
outp_key = ['1A', '2A', '3A', '4A', '5A', '6A', '7A', '8A', '9A', '10A', '11A', '12A', '1B', '2B', '3B', '4B', '5B', '6B', '7B', '8B', '9B', '10B', '11B', '12B']

#for genre purpose
targets = np.array([isi-1 for isi in targets])
print(targets.shape)
row_skf = [content for content in skf.split(x_train, y_train)]



(6640,)


In [28]:
inputs.shape

(6640, 65, 13)

In [31]:
# for index in range(10):
#     model, tl, tsl, acr = train_models(num_epochs, batch_size, learning_rate, x_train, y_train, row_skf, device, folds = index)
#     evaluation(model, val_loader,device, batch_size)
# # evaluation(model, ldr,device, batch_size)

model, tl, tsl, acr = train_models(num_epochs, batch_size, learning_rate, x_train, y_train, row_skf, device, folds = 0)
evaluation(model, val_loader,device, batch_size)

(10756, 65, 13) (10756,)
torch.Size([10756, 65, 13])
[1/30] Training loss: 1.3879	 Validation loss: 1.3356
[2/30] Training loss: 1.2820	 Validation loss: 1.3031
[3/30] Training loss: 1.2111	 Validation loss: 1.2640
[4/30] Training loss: 1.1423	 Validation loss: 1.1975
[5/30] Training loss: 1.0904	 Validation loss: 1.1760
[6/30] Training loss: 1.0642	 Validation loss: 1.1949
[7/30] Training loss: 1.0393	 Validation loss: 1.1658
[8/30] Training loss: 1.0177	 Validation loss: 1.1606
[9/30] Training loss: 1.0070	 Validation loss: 1.1533
[10/30] Training loss: 1.0073	 Validation loss: 1.1728
[11/30] Training loss: 1.0073	 Validation loss: 1.1765
[12/30] Training loss: 0.9997	 Validation loss: 1.1493
[13/30] Training loss: 0.9787	 Validation loss: 1.1321
[14/30] Training loss: 0.9573	 Validation loss: 1.1244
[15/30] Training loss: 0.9611	 Validation loss: 1.1217
[16/30] Training loss: 0.9548	 Validation loss: 1.1260
[17/30] Training loss: 0.9576	 Validation loss: 1.1352
[18/30] Training loss

In [41]:
model.eval()
torch.save(model.state_dict(), '81_3_genre_inf.pth')

In [40]:
#optional: export as ONNX model
model.eval()
batch_sizer = 1
x = torch.randn(batch_sizer, 65, 13, requires_grad=True).to(device)

torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "Genre__cls__81_3.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\onnx\symbolic.py:173: UserWarning: ONNX export failed on RNN/GRU/LSTM because batch_first not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


RuntimeError: ONNX export failed: Couldn't export operator aten::gru

Defined at:
<ipython-input-22-8e1b1a5a1eeb>(14): forward
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\nn\modules\module.py(481): _slow_forward
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\nn\modules\module.py(491): __call__
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\jit\__init__.py(294): forward
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\nn\modules\module.py(493): __call__
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\jit\__init__.py(231): get_trace_graph
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\onnx\utils.py(225): _trace_and_get_graph_from_model
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\onnx\utils.py(266): _model_to_graph
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\onnx\utils.py(363): _export
C:\Users\KRPTN\Anaconda3\envs\alphapose\lib\site-packages\torch\

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
predictions_one_hot = model.predict(inputs_test)
print(targets_test.shape, predictions_one_hot.shape)
# cm = confusion_matrix(targets_test, predictions_one_hot.argmax(axis=1))
# print(cm)
con_mat = tf.math.confusion_matrix(labels=targets_test, predictions=predictions_one_hot.argmax(axis=1)).numpy()
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = outp_key, 
                     columns = outp_key)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
signal, sr = librosa.load("../raw_dataset/files/BIG ROOM/01 Agus Zack _ Devbanz - Workout (Extended M.mp3") # len(signal) = 661794  # sr is 22050 by default 

In [ ]:
signal.shape[0]
mfcc = librosa.feature.mfcc(signal[0 : 22050*30],
                                                   sr = 22050,
                                                   n_fft = 4084,
                                                   n_mfcc = 13,
                                                   hop_length = 1024)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# (rate,sig) = wav.read("file.wav")
mfcc_feat = mfcc

ig, ax = plt.subplots()
mfcc_data= np.swapaxes(mfcc_feat, 0 ,1)
cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower', aspect='auto')
ax.set_title('MFCC')
#Showing mfcc_data
plt.show()
#Showing mfcc_feat
plt.plot(mfcc_feat)
plt.show()

In [ ]:
mfcc.shape